In [ ]:
#pip install sklearn #bei Nutzung mit Kaggle relevant
#pip install flair==0.6.1 #bei Nutzung mit Kaggle relevant
#!conda install -y gdown #bei Nutzung mit Kaggle relevant

In [6]:
import numpy as np 
import pandas as pd 
import json
import flair, torch
import os

In [7]:
#Standart Code von Kaggle.com

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os

#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [8]:
#flair.device = torch.device('cuda')

In [9]:
filename = "Data/train_data_ctxt_preliminary.json"

with open(filename,'r', encoding='utf8') as infile:
    example_data = json.load(infile)


for i,(k,v) in enumerate(example_data.items()):
    tokens = v.get('tokens')
    tokens = [token.lower() for token in tokens]
    example_data[k]['tokens'] = tokens
    
all_tokens = [v['tokens'] for k,v in example_data.items()]
all_labels = [v['labels'] for k,v in example_data.items()]

In [10]:
np.random.seed(1234567890)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(all_tokens, all_labels, test_size=0.2, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2

In [11]:
print(len(X_train), len(y_train))
print(len(X_test), len(y_test))
print(len(X_val), len(y_val))

4421 4421
1474 1474
1474 1474


In [12]:
import os

if os.path.exists("TrainTestVal/all.txt"):
  os.remove("all.txt")
else:
  print("The file does not exist")

f = open("TrainTestVal/all.txt", "x")

for x in range(0, len(all_tokens)):
    for y in range(0, len(all_tokens[x])):
        new_line = all_tokens[x][y] + " " + all_labels[x][y] + "\n"
        f.write(new_line)
    f.write("\n")
    
f.close()

if os.path.exists("TrainTestVal/train.txt"):
  os.remove("TrainTestVal/train.txt")
else:
  print("The file does not exist")

f = open("TrainTestVal/train.txt", "x")

for x in range(0, len(X_train)):
    for y in range(0, len(X_train[x])):
        new_line = X_train[x][y] + " " + y_train[x][y] + "\n"
        f.write(new_line)
    f.write("\n")
    
f.close()



if os.path.exists("TrainTestVal/test.txt"):
  os.remove("TrainTestVal/test.txt")
else:
  print("The file does not exist")

f = open("TrainTestVal/test.txt", "x")

for x in range(0, len(X_test)):
    for y in range(0, len(X_test[x])):
        new_line = X_test[x][y] + " " + y_test[x][y] + "\n"
        f.write(new_line)
    f.write("\n")
    
f.close()

if os.path.exists("TrainTestVal/val.txt"):
  os.remove("TrainTestVal/val.txt")
else:
  print("The file does not exist")

f = open("TrainTestVal/val.txt", "x")

for x in range(0, len(X_val)):
    for y in range(0, len(X_val[x])):
        new_line = X_val[x][y] + " " + y_val[x][y] + "\n"
        f.write(new_line)
    f.write("\n")
    
f.close()


The file does not exist
The file does not exist
The file does not exist
The file does not exist


In [13]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus

# define columns
columns = {0: 'text', 1: 'ner'}

# this is the folder in which train, test and dev files reside
data_folder = './'

# init a corpus using column format, data folder and the names of the train, dev and test files
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file='TrainTestVal/train.txt',
                              test_file='TrainTestVal/test.txt',
                              dev_file='TrainTestVal/val.txt')

2020-10-22 11:50:48,375 Reading data from .
2020-10-22 11:50:48,377 Train: TrainTestVal/train.txt
2020-10-22 11:50:48,378 Dev: TrainTestVal/val.txt
2020-10-22 11:50:48,379 Test: TrainTestVal/test.txt


In [14]:
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings, TransformerWordEmbeddings, FlairEmbeddings, CharacterEmbeddings, OneHotEmbeddings
tag_type = 'ner'

# 3. make the tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)
print(tag_dictionary)



Dictionary with 20 tags: <unk>, O, B_C, I_C, B_E, B_T, B_P, I_P, B_M, I_M, B_V, I_V, I_T, B_W, I_E, B_A, I_A, I_W, <START>, <STOP>


In [ ]:
#Hier können die Hyperparameter konfiguriert werden
#ein oder mehrere Embeddings, Language Models einfach auskommentieren
#Hidden size der LSTM Layer kann definiert werden
#Layer Anzahl kann definiert werden
#CRF Layer

embedding_types = [

    WordEmbeddings('glove'),
    # TransformerWordEmbeddings("bert-base-uncased"),

    #CharacterEmbeddings(),

    # comment in these lines to use flair embeddings
    FlairEmbeddings('news-forward'),
    FlairEmbeddings('news-backward')
    #ELMoEmbeddings('medium')
]

embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)


# 5. initialize sequence tagger
from flair.models import SequenceTagger

tagger: SequenceTagger = SequenceTagger(hidden_size=128, #128, 256, 512
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        #loss_weights = weight_dict,
                                        rnn_layers = 2, #Number of rnn_layers
                                        tag_type=tag_type,
                                        use_crf=True)

# 6. initialize trainer
from flair.trainers import ModelTrainer

trainer: ModelTrainer = ModelTrainer(tagger, corpus)
    
    
    
# 7. start training
trainer.train('resources/taggers/NewModel',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=150,
             embeddings_storage_mode = "gpu")#oder cpu


    

2020-10-22 11:50:54,659 ----------------------------------------------------------------------------------------------------
2020-10-22 11:50:54,660 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings('glove')
    (list_embedding_1): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.05, inplace=False)
        (encoder): Embedding(300, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=300, bias=True)
      )
    )
    (list_embedding_2): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.05, inplace=False)
        (encoder): Embedding(300, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=300, bias=True)
      )
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=4196, out_features=4196, bias=True)
  (rnn): LSTM(4196, 128, num_layers=2, batch

In [ ]:
from flair.data import Sentence

# load the model you trained
model = SequenceTagger.load('resources/taggers/NewModel/best-model.pt')


# create example sentence
sentence = Sentence('friendly people were here for a birthday Party')

# predict tags and print
model.predict(sentence)

print(sentence.to_tagged_string())